Known issues: <br>
-Lots of soundfonts don't play on FluidSynth, but works in a sfz/sf2 player. This includes: <br>
drums not working <br>
bass guitar not working (currently substituting a regular acoustic guitar for the bass sound) <br>

-Gain not adjustable, doesn't seem like midi2audio can pass gain options to FluidSynth <br>


In [1]:
from midi2audio import FluidSynth
from mido import MidiFile

In [2]:
#For current working directory
import os 
cwd = os.getcwd()
cwd

'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception'

In [3]:
# Default filepath and soundfonts
filepath = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\MIDI\\seven_nation_army+1.mid'
soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2'    
fs = FluidSynth(sound_font=soundfont) #create fs object with designated soundfont

In [ ]:
# Play MIDI using selected soundfont above
FluidSynth(sound_font=soundfont).play_midi(track_1)

# Splitting MIDI

#### MIDI split function

In [135]:
def midi_to_audio(input_midi):
    try:
        # Initialize a list to hold the individual track variables
        tracks = []
        
        # Initialize program_number for use later
        program_number = None
        
        # Load MIDI file using mido.MidiFile
        mid = MidiFile(input_midi)

        # Iterate through each MIDI track and save as .mid and varaiable
        for i, track in enumerate(mid.tracks):
            # Save tracks to separate MIDI files
            output_midi = f"track_{i}.mid"
            mid_new = MidiFile(ticks_per_beat=mid.ticks_per_beat)
            mid_new.tracks.append(track)
            mid_new.save(output_midi)
            
            # Store tracks to separate variables
            globals()[f'track_{i}'] = mid_new
            
            # Print confirmation
            print(f"Track {i} saved as {output_midi} and as the variable track_{i}")
            
            
            # Check through each track for the program number            
            for msg in mid_new:
                if msg.type == 'program_change':
                    program_number = msg.program
                    break
                
            # Determine which instrument to use. If program is None then skip
            if program_number is not None:
                soundfont = timbre(program_number)
                fs = FluidSynth(sound_font=soundfont)
                
                # Output to wav file
                output_file = f"track_{i}.wav"
                fs.midi_to_audio(f'track_{i}.mid', output_file)
                # Print confirmation
                print(f"Track {i} saved as track_{i}.wav")
        
    except Exception as e:
        print(f"Error: {e}")

In [158]:
# Use program number to determine which instrument soundfont to use
def timbre(program_number):

    if program_number >=0 and program_number <=7: #Piano Timbres in MIDI are programs 0-7
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano_steinway_grand.sf2'
        
    elif program_number >=8 and program_number <=15: #Chromatic Percussion in MIDI are programs 8-15
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\glockenspiel.sf2' 
        
#     elif program_number >=16 and program_number <=23: #Organ Timbres in MIDI are programs 16-23
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2' 

    elif program_number >=24 and program_number <=31: #Guitar Timbres in MIDI are programs 24-31
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\guitar_universal.sf2' 

    elif program_number >=32 and program_number <=39: #Bass Timbres in MIDI are programs 32-39
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\guitar_universal.sf2' 
        
#     elif program_number >=40 and program_number <=47: #String Timbres in MIDI are programs 40-47
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\cello.sf2'

    elif program_number >=48 and program_number <=55: #Ensemble Timbres in MIDI are programs 48-55
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\strings_violin.sf2'

#     elif program_number >=56 and program_number <=63: #Brass Timbres in MIDI are programs 56-63
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2'

#     elif program_number >=64 and program_number <=71: #Reed Timbres in MIDI are programs 64-71
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2'

    elif program_number >=72 and program_number <=79: #Pipe Timbres in MIDI are programs 72-79
        soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\flute.sf2'

#     elif program_number >=80 and program_number <=87: #Synth Lead in MIDI are programs 80-87
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2'

#     elif program_number >=88 and program_number <=95: #Synth Pad in MIDI are programs 88-95
#         soundfont = 'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception\\Soundfonts\\piano.sf2'

    else:
        print(f"Error: Timbre not yet supported")
        
    return soundfont

In [149]:
#Testing
input_midi = 'MIDI\\honestly_result_flute.mid'  # Replace with your input MIDI file
midi_to_audio(input_midi)

Track 0 saved as track_0.mid and as the variable track_0
Track 1 saved as track_1.mid and as the variable track_1
Track 1 saved as track_1.wav
Track 2 saved as track_2.mid and as the variable track_2
Track 2 saved as track_2.wav
Track 3 saved as track_3.mid and as the variable track_3
Track 3 saved as track_3.wav
Track 4 saved as track_4.mid and as the variable track_4
Track 4 saved as track_4.wav
Track 5 saved as track_5.mid and as the variable track_5
Track 5 saved as track_5.wav
Track 6 saved as track_6.mid and as the variable track_6
Track 6 saved as track_6.wav


### Try combining into one file

In [150]:
import numpy as np
from scipy.io.wavfile import read, write
from IPython.display import Audio

In [151]:
(fs, x1) = read('track_1.wav')
(fs, x2) = read('track_2.wav')
(fs, x3) = read('track_3.wav')
(fs, x4) = read('track_4.wav')
(fs, x5) = read('track_5.wav')
(fs, x6) = read('track_6.wav')

In [152]:
x1 = x1[:,0]
x2 = x2[:,0]
x3 = x3[:,0]
x4 = x4[:,0]
x5 = x5[:,0]
x6 = x6[:,0]

In [153]:
# max_length = max(len(x1), len(x2), len(x3), len(x4), len(x5))
max_length = max(len(x1), len(x2), len(x3), len(x4), len(x5), len(x6))
max_length

1098752

In [154]:
padded_x1 = np.pad(x1, (0, max_length - len(x1)), mode='constant')
padded_x2 = np.pad(x2, (0, max_length - len(x2)), mode='constant')
padded_x3 = np.pad(x3, (0, max_length - len(x3)), mode='constant')
padded_x4 = np.pad(x4, (0, max_length - len(x4)), mode='constant')
padded_x5 = np.pad(x5, (0, max_length - len(x5)), mode='constant')
padded_x6 = np.pad(x6, (0, max_length - len(x6)), mode='constant')

In [155]:
# sum = padded_x1 + padded_x2 + padded_x3 + padded_x4 + padded_x5 
sum = padded_x1 + padded_x2 + padded_x3 + padded_x4 + padded_x5 + padded_x6

In [156]:
Audio(sum, rate=fs)

In [157]:
write('honestly_result_flute.wav', fs, sum)